# Composite Types

In [2]:
struct MyVec1
    x
    y
end

In [3]:
v = MyVec1(1, 2)

MyVec1(1, 2)

In [4]:
v.x

1

In [5]:
struct MyVec2
    x::Float64
    y::Float64
end

In [6]:
MyVec2(1, 2)

MyVec2(1.0, 2.0)

# Parameters

In [8]:
struct MyVec3{T}
    x::T
    y::T
end

In [9]:
MyVec3(1.0, 2.0)

MyVec3{Float64}(1.0, 2.0)

In [10]:
v = MyVec3(1, 2)

MyVec3{Int64}(1, 2)

In [11]:
v.x += 2

LoadError: setfield!: immutable struct of type MyVec3 cannot be changed

# Mutability

In [12]:
mutable struct MyVec4{T}
    x::T
    y::T
end

In [13]:
v = MyVec4(1, 2)

MyVec4{Int64}(1, 2)

In [14]:
v.x += 2

3

In [15]:
v

MyVec4{Int64}(3, 2)

**Major Downside** of mutability: mutable objects are usually allocated on the heap.

# Methods

In [16]:
function myplus(a::MyVec3, b::MyVec3)
    return MyVec3(a.x + b.x, a.y + b.y)
end

myplus (generic function with 1 method)

In [17]:
myplus(MyVec3(1,2), MyVec3(3,4))

MyVec3{Int64}(4, 6)

# Abstract Types

In [19]:
"""
An AbstractVec represents a 2-d vector.

New AbstractVec types should implement the methods horizontal(v) and vertical(v) to get the x and y coordinates
"""
abstract type AbstractVec end

AbstractVec

In [20]:
struct Vec{T} <: AbstractVec
    x::T
    y::T
end

horizontal(v::Vec) = v.x
vertical(v::Vec) = v.y

vertical (generic function with 1 method)

In [21]:
mutable struct MVec{T} <: AbstractVec
    x::T
    y::T
end

horizontal(v::MVec) = v.x
vertical(v::MVec) = v.y

vertical (generic function with 2 methods)

In [22]:
function myplus(a::AbstractVec, b::AbstractVec)
    return typeof(a)(horizontal(a) + horizontal(b), vertical(a) + vertical(b))
end

myplus (generic function with 2 methods)

In [23]:
myplus(Vec(1,2), MVec(3,4))

Vec{Int64}(4, 6)

In [24]:
struct ZeroVec <: AbstractVec end

horizontal(::ZeroVec) = 0.0
vertical(::ZeroVec) = 0.0

vertical (generic function with 3 methods)

In [25]:
myplus(Vec(1,2), ZeroVec())

Vec{Int64}(1, 2)

# Adding methods to functions from another module

In [26]:
@doc zero

```
zero(x)
zero(::Type)
```

Get the additive identity element for the type of `x` (`x` can also specify the type itself).

See also [`iszero`](@ref), [`one`](@ref), [`oneunit`](@ref), [`oftype`](@ref).

# Examples

```jldoctest
julia> zero(1)
0

julia> zero(big"2.0")
0.0

julia> zero(rand(2,2))
2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
```


In [27]:
zero(Vec(1,2))

LoadError: MethodError: no method matching zero(::Vec{Int64})
[0mClosest candidates are:
[0m  zero([91m::Union{Type{P}, P}[39m) where P<:Dates.Period at /opt/julia-1.7.1/share/julia/stdlib/v1.7/Dates/src/periods.jl:53
[0m  zero([91m::LinearAlgebra.UniformScaling{T}[39m) where T at /opt/julia-1.7.1/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:136
[0m  zero([91m::AbstractIrrational[39m) at /opt/julia-1.7.1/share/julia/base/irrationals.jl:150
[0m  ...

In [28]:
Base.zero(v::Vec) = Vec(0,0)
Base.zero(v::MVec) = MVec(0,0)
Base.zero(v::ZeroVec) = ZeroVec()

In [29]:
zero(Vec(1,2))

Vec{Int64}(0, 0)

In [30]:
Base.:+(a::AbstractVec, b::AbstractVec) = typeof(a)(horizontal(a) + horizontal(b), vertical(a) + vertical(b))
Base.:+(a::ZeroVec, b::ZeroVec) = ZeroVec()

In [31]:
function Base.:*(a::Int, b::AbstractVec)
    c = zero(b)
    for i in 1:a
        c += b
    end
    return c
end

In [32]:
2 * Vec(1, 1)

Vec{Int64}(2, 2)

In [33]:
2*MVec(1, 2)

MVec{Int64}(2, 4)

In [34]:
2*ZeroVec()

ZeroVec()

## Creating a Policy

In [1]:
using POMDPs